### Example notebook

In [ ]:
import matplotlib.pyplot as plt

from pommes import test_case_path
from pommes.io.build_input_dataset import (
    build_input_parameters,
    read_config_file,
)
from pommes.io.save_solution import save_solution
from pommes.model.build_model import build_model
from pommes.model.data_validation.dataset_check import check_inputs

In [ ]:
scenario = "ref"
suffix = "_02161113"

output_folder = f"{test_case_path}/output/{scenario}{suffix}"
solver = "highs"  # ["gurobi", "xpress", "highs", "mosek"]

In [ ]:
config = read_config_file(file_path=test_case_path / "config.yaml")

model_parameters = build_input_parameters(config)
model_parameters = check_inputs(model_parameters)

In [ ]:
model = build_model(model_parameters)

In [ ]:
model.solve(solver_name=solver)

save_solution(
    model=model,
    output_folder=output_folder,
    model_parameters=model_parameters,
)

In [ ]:
solution = model.solution
print(model.solution)

In [ ]:
dual = model.dual
print(dual)

In [ ]:
prices = dual.operation_adequacy_constraint
df = prices.to_dataframe(name="value").reset_index()
print(df)

In [ ]:
unique_resources = df["resource"].unique()
unique_years = df["year_op"].unique()
unique_areas = df["area"].unique()
colors = plt.cm.tab10(range(len(unique_areas)))
area_color_map = {area: colors[i] for i, area in enumerate(unique_areas)}

fig, axes = plt.subplots(
    len(unique_resources),
    len(unique_years),
    figsize=(15, 10),
    sharex=True,
    sharey="row",
)

for i, resource in enumerate(unique_resources):
    for j, year in enumerate(unique_years):
        ax = axes[i, j]
        subset = df[(df["resource"] == resource) & (df["year_op"] == year)]

        for area in unique_areas:
            area_data = subset[subset["area"] == area]
            if not area_data.empty:
                ax.plot(
                    area_data["hour"],
                    area_data["value"],
                    marker="o",
                    color=area_color_map[area],
                )
        if i == 0:
            ax.set_title(f"Year: {year}")
        if j == 0:
            ax.set_ylabel(f"Resource: {resource}")
        if i == len(unique_resources) - 1:
            ax.set_xlabel("Hour")

legend_elements = [
    plt.Line2D(
        [0],
        [0],
        marker="o",
        color=color,
        linestyle="",
        markersize=8,
        label=area,
    )
    for area, color in area_color_map.items()
]
fig.legend(
    handles=legend_elements,
    loc="upper center",
    ncol=len(unique_areas),
    title="Area",
)

plt.tight_layout(rect=(0, 0, 1, 0.95))
plt.show()